# Introductory applied machine learning (INFR10069)

# Assignment 4: Feature Engineering

## Marking Breakdown

**70-100%** results/answer correct plus extra achievement at understanding or analysis of results. Clear explanations, evidence of creative or deeper thought will contribute to a higher grade.

**60-69%** results/answer correct or nearly correct and well explained.

**50-59%** results/answer in right direction but significant errors.

**40-49%** some evidence that the student has gained some understanding, but not answered the questions
properly.

**0-39%** serious error or slack work.


## Mechanics

You should produce a Jupyter notebook in answer to this assignment.
**You need to submit this notebook electronically as described below.**

Place your notebook in a directory called `iamlans` and submit this directory using the submit command on a DICE machine. The format is:

`submit iaml 4 iamlans`

You can check the status of your submissions with the `show_submissions` command.

**Late submissions:** The policy stated in the School of Informatics MSc Degree Guide is that normally you will not be allowed to submit coursework late. See http://www.inf.ed.ac.uk/teaching/years/msc/courseguide10.html#exam for exceptions to this, e.g. in case of serious medical illness or serious personal problems.

**Collaboration:** You may discuss the assignment with your colleagues, provided that the writing that you submit is entirely your own. That is, you should NOT borrow actual text or code from other students. We ask that you provide a list of the people who you've had discussions with (if any).

## Important Instructions

1. In the following questions you are asked to run experiments using Python (version 2.7) and the following packages:
    * Numpy
    * Pandas
    * Scikit-learn 0.17
    * Matplotlib
    * Seaborn

2. Before you start make sure you have set up a vitual environment (or conda environment if you are working on your own machine) and the required packages installed. Instructions on how to set-up the working enviornment and install the required packages can be found in `01_Lab_1_Introduction`.

3. Wherever you are required to produce code you should use code cells, otherwise you should use markdown cells to report results and explain answers. **You are welcome to split your answer into multiple cells with intermediate printing.**

4. The .csv files that you will be using are located at `./datasets` (the `datasets` directory is adjacent to this file).

5. **IMPORTANT:** Keep your answers brief and concise. Most questions can be answered with 2-3 lines of explanation (excluding coding questions), unless stated otherwise.

## Imports

In this assignment you are asked to import all the packages and modules you will need. Include all required imports and execute the cell below.

In [1]:
from __future__ import division #print_function
import os
import math
from collections import OrderedDict
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.svm import LinearSVC, SVC
from sklearn.cross_validation import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.naive_bayes import GaussianNB
%matplotlib inline

/home/student/anaconda2/envs/iaml/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
seed = 0
rng = np.random.RandomState(seed=seed)

## Description of the datasets


This assignment is based on two datasets:
1. the 20 Newsgroups Dataset (you should recognise it from Assignment 1)
2. the MNIST digits dataset

### 20 Newsgroups

For convenience, we repeat the description here. This dataset is a collection of approximately **20,000 newsgroup documents**, partitioned (nearly) evenly across 20 different newsgroups, each corresponding to a different topic. Some of the newsgroups are **very closely related** to each other (e.g. comp.sys.ibm.pc.hardware, comp.sys.mac.hardware), while others are **highly unrelated** (e.g misc.forsale, soc.religion.christian). 

To save you time and to make the problem manageable with limited computational resources, we preprocessed the original dataset. We will use documents from only **5 out of the 20 newsgroups**, which results in a 5-class problem. More specifically the 5 classes correspond to the following newsgroups: 
1. `alt.atheism`
2. `comp.sys.ibm.pc.hardware`
3. `comp.sys.mac.hardware`
4. `rec.sport.baseball`
5. `rec.sport.hockey `

However, note here that classes **2-3** and **4-5** are rather closely related.

**In contrast to Assignment 1**, we have opted to use **tf-idf** weights ([term frequency - inverse document frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf))
for each word instead of the frequency counts. These weights represent the importance of a word to a
document with respect to a collection of documents. The importance increases proportionally to the number
of times a word appears in the document and decreases proportionally to the number of times the word
appears in the whole corpus. 

Additionally we preprocess the data to include the **most frequent 1000 words** that are in **greater than 2 documents**, less than half of all documents, and that are not **[stop words](https://en.wikipedia.org/wiki/Stop_words)**.

We will perform all this preprocessing for you.


### MNIST
This MNIST Dataset is a collection handwritten digits. The samples are partitioned (nearly) evenly across the **10 different digit classes {0, 1, . . . , 9}**. We use a preprocessed version for which the data are **$8 \times 8$** pixel images containing one digit each. For further details on how the digits are preprocessed, see the sklearn documentation. The **images are grayscale**, with each pixel taking values in **{0, 1, . . . , 16}**, where 0 corresponds to black (weakest intensity) and 16 corresponds to white (strongest intensity). Therefore, the dataset is a **N × 64** dimensional matrix where each dimension corresponds to a pixel from the image and N is the number of
images. 

Again, to save you time, we perfom the import for you.

In [48]:
X = np.array([
             [4, 0],
             [1, math.sqrt(3)],
             [1, -math.sqrt(3)],
             [-math.sqrt(3), 3],
             [math.sqrt(3), 3],
             [0, 0]
    ])

print X.shape
X

(6, 2)


array([[ 4.        ,  0.        ],
       [ 1.        ,  1.73205081],
       [ 1.        , -1.73205081],
       [-1.73205081,  3.        ],
       [ 1.73205081,  3.        ],
       [ 0.        ,  0.        ]])

In [49]:
y = np.array([0, 0, 0, 1, 1, 1])

In [50]:
def get_p(mu,var):
    return lambda x : math.exp(- math.pow(x - mu, 2) / (2*var) )

In [51]:
def train(X, y, log=False):
    Pgenuine = len(y[y==0]) / len(y)
    if log:
        print "Pgenuine: " + Pgenuine
    Pintruder = len(y[y==1]) / len(y)

    if log:
        print "Pintruder: " + Pintruder
        
    mu_ls_genuine = np.mean(X[y==0, 0])
    var_ls_genuine = np.var(X[y==0, 0])
    #mu_ls_genuine, var_ls_genuine
    mu_ls_intruder = np.mean(X[y==1, 0])
    var_ls_intruder = np.var(X[y==1, 0])
    #mu_ls_intruder, var_ls_intruder
    mu_cd_genuine = np.mean(X[y==0, 1])
    var_cd_genuine = np.var(X[y==0, 1])
    #mu_cd_genuine, var_cd_genuine
    mu_cd_intruder = np.mean(X[y==1, 1])
    var_cd_intruder = np.var(X[y==1, 1])
    #mu_cd_intruder, var_cd_intruder
    
    p_ls_genuine = get_p(mu_ls_genuine, var_ls_genuine)
    p_ls_intruder = get_p(mu_ls_intruder, var_ls_intruder)
    p_cd_genuine = get_p(mu_cd_genuine, var_cd_genuine)
    p_cd_intruder = get_p(mu_cd_intruder, var_cd_intruder)
    
    p_intruder = lambda ls, cd : Pintruder * p_ls_intruder(ls) * p_cd_intruder(cd)
    p_genuine = lambda ls, cd : Pgenuine * p_ls_genuine(ls) * p_cd_genuine(cd)
    
    return p_intruder, p_genuine

In [52]:
def test(ls, cd, log=False):
    intruder = p_intruder(ls, cd)
    genuine = p_genuine(ls, cd)
    
    if log:
        print "intruder: %f" % intruder
        print "genuine: %f" % genuine
    
    if genuine > intruder:
        return 0
    elif intruder > genuine:
        return 1
    else: return -1

In [53]:
p_intruder, p_genuine = train(X, y)
test(0, 0, True)

intruder: 0.183940
genuine: 0.183940


-1

In [54]:
def getPreds(X, y):
    p_intruder, p_genuine = train(X, y)
    return np.array([test(x[0], x[1]) for x in X])

In [55]:
getPreds(X, y)

array([ 0,  1,  0,  1,  1, -1])

In [56]:
getPreds(X[1:], y[1:])

array([ 1,  0,  1,  1, -1])

In [57]:
getPreds(np.delete(X, 1, axis=0), np.delete(y, 1))

array([ 0,  0,  1,  1, -1])

In [58]:
getPreds(np.delete(X, 2, axis=0), np.delete(y, 2))

array([ 0,  1,  1,  1, -1])

In [59]:
getPreds(np.delete(X, 3, axis=0), np.delete(y, 3))

array([ 0,  1,  0,  1, -1])

In [60]:
getPreds(np.delete(X, 4, axis=0), np.delete(y, 4))

array([ 0,  1,  0,  1, -1])

In [61]:
getPreds(np.delete(X, 5, axis=0), np.delete(y, 5))

array([0, 1, 0, 1, 1])